In [ ]:
import pandas as pd
import os
from collections import defaultdict
from numpy import nan

all_files = [f for f in os.listdir('../alldrugbank/')]

results_dir = '../alldrugbank'
files_to_analyze = []
file_ext = '_merged_neighborhood_.txt'
for (dir_name,sub_dir,file_list) in os.walk(results_dir):
    for file_name in file_list:
        if file_ext in file_name:
            files_to_analyze.append(file_name)

# Obtains all DrugBank ID's so we are able to get the path to the tables
drugbank_ids = []
for f in range(0, len(files_to_analyze)-1):
    identification = files_to_analyze[f].split('_')[0]
    drugbank_ids.append(identification)

path = '../alldrugbank'

drugs_to_genes = defaultdict(set)

# Create default dictionary with drugBankID to list of network genes!
for f in range(0, len(files_to_analyze)-1):
    network_genes = []
    if os.stat(os.path.join(path, drugbank_ids[f], files_to_analyze[f])).st_size == 0:
        continue
    rfile = pd.read_csv(os.path.join(path, drugbank_ids[f], files_to_analyze[f]),sep='\t', header=None)
    for i in rfile[0]:
        if pd.isnull(i):
            continue
        network_genes.append(i)
    for j in rfile[1]:
        if pd.isnull(j):
            continue
        network_genes.append(j)
    for x in network_genes:
        if pd.isnull(x):
            network_genes.remove(x)
    network_genes = set(network_genes)
    drugs_to_genes[drugbank_ids[f]] = network_genes

import pandas as pd
# Read in csv that has drugbank ID and its common names and synonyms
rfile = '../drugbank_vocabulary.csv'
drugvocab = pd.read_csv(rfile,sep=',')

df = pd.read_excel('../sweetlead_ranked.xlsx')
df = df.fillna(method='ffill')

# Mines each of the drug names and puts all their names into a list of cleaned up names
cleaned_up_names = []
for i in range(0,len(df["OfficialNames"])):
    all_countries = df["OfficialNames"][i].split(',')
    for country in all_countries:
        if ';' in country:
            equal_sign = country.find("=")
            drugnames = country[equal_sign+1:]
            x = drugnames.split(';')
            for i in x:
                cleaned_up_names.append(i)
        else:
            equal_sign = country.find("=")
            drugnames = country[equal_sign+1:]
            cleaned_up_names.append(drugnames)

for i in range(0,len(cleaned_up_names)-1):
    cleaned_up_names[i] = cleaned_up_names[i].title()

unique_names = set(cleaned_up_names)

unique_names_list = list(unique_names)
unique_names_list.sort()

drugs_to_values = defaultdict(int)
drugbank_ids_list = []

for i in range(0,len(unique_names_list)-1):
    for j in drugvocab["Common name"]:
        if unique_names_list[i] == j:
            x = drugvocab.loc[drugvocab["Common name"] == unique_names_list[i], "DrugBank ID"].values
            x = ''.join(x)
            drugbank_ids_list.append(x)

for f in range(0, len(drugbank_ids_list)-1):
    drugs_to_values[drugbank_ids_list[f]] = 1

df2 = pd.read_excel('../sweetlead_ranked.xlsx')

df2 = df2.fillna(method='ffill')

cleaned_up_names2 = []
for i in range(0,len(df2["OfficialNames"])):
    all_countries = df2["OfficialNames"][i].split(',')
    for country in all_countries:
        if ';' in country:
            equal_sign = country.find("=")
            drugnames = country[equal_sign+1:]
            x = drugnames.split(';')
            for i in x:
                cleaned_up_names2.append(i)
        else:
            equal_sign = country.find("=")
            drugnames = country[equal_sign+1:]
            cleaned_up_names2.append(drugnames)

for i in range(0,len(cleaned_up_names2)-1):
    cleaned_up_names2[i] = cleaned_up_names2[i].title()

unique_names2 = set(cleaned_up_names2)

unique_names_list2 = list(unique_names2)
unique_names_list2.sort()

drugs_to_values2 = defaultdict(int)
drugbank_ids_list2 = []

for i in range(0,len(unique_names_list2)-1):
    for j in drugvocab["Common name"]:
        if unique_names_list2[i] == j:
            x = drugvocab.loc[drugvocab["Common name"] == unique_names_list2[i], "DrugBank ID"].values
            x = ''.join(x)
            drugbank_ids_list2.append(x)

for f in range(0, len(drugbank_ids_list2)-1):
    drugs_to_values2[drugbank_ids_list2[f]] = 0

drugs_to_values.update(drugs_to_values2)

In [ ]:
drugs_to_values

defaultdict(int,
            {'DB00459': 1,
             'DB01143': 1,
             'DB01025': 1,
             'DB01125': 1,
             'DB00335': 1,
             'DB01117': 1,
             'DB00436': 1,
             'DB00562': 1,
             'DB00612': 1,
             'DB01194': 1,
             'DB00482': 1,
             'DB00369': 1,
             'DB00537': 1,
             'DB09131': 1,
             'DB00694': 1,
             'DB06782': 1,
             'DB00869': 1,
             'DB00997': 1,
             'DB00651': 1,
             'DB00467': 1,
             'DB00668': 1,
             'DB00445': 1,
             'DB00330': 1,
             'DB00898': 1,
             'DB00311': 1,
             'DB00798': 1,
             'DB00801': 1,
             'DB13322': 1,
             'DB01050': 1,
             'DB09487': 1,
             'DB00893': 1,
             'DB01064': 1,
             'DB06218': 1,
             'DB00583': 1,
             'DB00281': 1,
             'DB00978': 1,
           

In [ ]:
all_rows = []
for (dbid, effect_size) in drugs_to_values.items():
    net_genes = drugs_to_genes[dbid]
    row_data = {'name':dbid, 'effect':effect_size}
    for g in net_genes:
        row_data[g]=1
    all_rows.append(row_data)

df = pd.DataFrame(all_rows)
df.fillna(0)

,name,effect,KRT1,rs2727786,rs62576288,CTSL,rs2854116,KRT6B,NCOR2,RARG,...,rs5182,rs5186,rs275651,rs12721226,CACNA1A,RBP2,LRAT,rs1741981,LEF1,HDAC1
0,DB00459,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DB01143,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DB01025,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DB01125,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DB00335,1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,DB00177,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
281,DB00661,0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
282,DB00194,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
283,DB00162,0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
df = df.fillna(0)

In [ ]:
df.to_csv("df.csv")

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
import pickle,os,csv,math
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from optparse import OptionParser

In [ ]:
# mapping for output
out_map = ['Increase', 'Decrease']

# save drugnames and drug banks ids but remove them for the regression
dbids = df['name'].to_list()
df = df.drop(['name'],axis=1)
df = df.drop(['Unnamed: 0',],axis=1)

y = df.loc[:,'effect']
X = df.drop(['effect'],axis=1)
gnames = list(X.columns)

split_row = math.ceil(X.shape[0]*0.2) # save 20% of rows for testing
X_test = X.loc[:split_row,:]
y_test = y.loc[:split_row,]
X_train = X.loc[split_row:,:]
y_train = y.loc[split_row:,]

model = LogisticRegression(random_state=0)
model.fit(X,y)
predicted_classes = model.predict(X)

print('Mean squared error: %.2f'% mean_squared_error(y, predicted_classes))
print('Coefficient of determination: %.2f'% r2_score(y, predicted_classes))

Mean squared error: 0.12
Coefficient of determination: 0.40


In [ ]:
reg_coef = list(zip(gnames, model.coef_[0]))
sort_rc = sorted(reg_coef,key = lambda x:x[1],reverse=True)
out_df = pd.DataFrame({'Gene Names':gnames,'Regression Coefficients':model.coef_[0]})
out_df = out_df.sort_values(by='Regression Coefficients', ascending=False)
out_df.to_excel('insilico_phagocyt_logregcoeffs.xlsx',index=False)

In [ ]:
outf = "drugScreenLogisticRegression.xlsx"
writer = pd.ExcelWriter(outf,engine='xlsxwriter')
reg_coef = list(zip(gnames, model.coef_[0]))
sort_rc = sorted(reg_coef,key = lambda x:x[1],reverse=True)
out_df = pd.DataFrame({'Gene Names':gnames,'Regression Coefficients':model.coef_[0]})
out_df = out_df.sort_values(by='Regression Coefficients', ascending=False)
out_df.to_excel(writer,sheet_name=outf,index=False)
writer.save()